### Objectif: 
#### Génération de datasets avec comme feature les verbes d'action du document, en utilisant les datasets précédemment construits et en enlevant tous les autres mots des features

In [1]:
import treetaggerwrapper as tt
tt_fr = tt.TreeTagger(TAGLANG='fr',TAGDIR='C:/TreeTagger')
import pandas as pd 
import csv
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from difflib import SequenceMatcher

def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

def is_action_verb(verbe):
    with open('../Ressources/verbes_cs.txt') as actions:
        for action in actions:
            if(similar(verbe.upper(), action) > 0.8):
                return True                        
    return False

def matrix_with_verbs():
    train_data = pd.read_csv('../Ressources/separated_data/scenarii_training_text.csv', encoding="ISO-8859-1")
    train_tags = pd.read_csv('../Ressources/separated_data/scenarii_training_tags.csv', encoding="ISO-8859-1")
    test_data = pd.read_csv('../Ressources/separated_data/scenarii_testing_text.csv', encoding="ISO-8859-1")
    test_tags = pd.read_csv('../Ressources/separated_data/scenarii_testing_tags.csv', encoding="ISO-8859-1")

    train_data = train_data[train_data.columns[0]]
    train_tags = train_tags[train_tags.columns[0]]
    test_data = test_data[test_data.columns[0]]
    test_tags = test_tags[test_tags.columns[0]]
    
    verbes = list()
    for sentence in train_data:
        tags = tt_fr.TagText(sentence)
        for word in tags:
            if(len(word.split("\t")) > 1):
                POS = word.split("\t")[1]
                word = word.split("\t")[0]
                if(("VER" in POS)):
                    if((word not in verbes)):
                        if(is_action_verb(word)):
                            verbes.append(word)
                            
    cv = CountVectorizer(vocabulary=verbes)
    matrix = cv.fit_transform(train_data).toarray()
    test_data = cv.fit_transform(test_data).toarray()
    return (matrix, train_tags, test_data, test_tags, cv.get_feature_names())

In [2]:
resultat = matrix_with_verbs()
with open('../Ressources/data_verbs_only/training_matrix_with_action_verbs.csv', 'w', newline='', encoding='ISO-8859-1') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(resultat[4])
    for line in resultat[0]:
        l = line.tolist()
        writer.writerow(l)
with open('../Ressources/data_verbs_only/training_tags_with_action_verbs.csv', 'w', newline='', encoding='ISO-8859-1') as csvfile:
    writer = csv.writer(csvfile)
    for line in resultat[1]:
        writer.writerow([line])
with open('../Ressources/data_verbs_only/testing_text_with_action_verbs.csv', 'w', newline='', encoding='ISO-8859-1') as csvfile:
    writer = csv.writer(csvfile)
    for line in resultat[2]:
        l = line.tolist()
        writer.writerow(l)
with open('../Ressources/data_verbs_only/testing_tags_with_action_verbs.csv', 'w', newline='', encoding='ISO-8859-1') as csvfile:
    writer = csv.writer(csvfile)
    for line in resultat[3]:
        writer.writerow([line])